In [35]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "01-03-4"

In [36]:
!pip install -qU langchain_core langchain_openai

### pydantic 사용하지 않을 때 (구조화)

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [38]:
model = ChatOpenAI(temperature=0, model_name = 'gpt-4o')

In [39]:
class Topic(BaseModel):
  description: str = Field(description='주제에 대한 간결한 설명')
  hashtags : str = Field(description = '해시태그 형식의 키워드 (2개 이상)')

In [40]:
parser = JsonOutputParser(pydantic_schema = Topic)

In [41]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "#Format: {format_instructions}\n\n #Qustion: {question}"),
    ]
)

prompt = prompt.partial(format_instructions = parser.get_format_instructions())

In [42]:
chain = prompt | model | parser

In [53]:
question = "유튜버의 폐해."
answer = chain.invoke({'question':question})

In [52]:
answer

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상입니다. 이는 주로 화석 연료의 연소, 산림 파괴, 산업 활동 등 인간의 활동에 의해 촉발됩니다. 지구 온난화는 기후 변화, 해수면 상승, 극단적인 기상 현상 증가 등 다양한 환경적 영향을 초래합니다.',
 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지속가능성']}

In [46]:
print(type(answer))

<class 'dict'>


### pydantic 사용하지 않을 때

In [32]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 'description'예, 관련 키워드는 'hashtags'에 담아주세요."

#JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬포트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "#Format: {format_instructions}\n\n #Qustion: {question}"),
    ]
)

# 지시사항을 프롬포트에 주입합니다.
prompt = prompt.partial(format_instructions = parser.get_format_instructions())

# 프롬포트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question" : question})

In [49]:
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상입니다. 이는 주로 화석 연료의 연소, 산림 파괴, 산업 활동 등 인간의 활동에 의해 촉발됩니다. 지구 온난화는 기후 변화, 해수면 상승, 극단적인 기상 현상 증가 등 다양한 환경적 영향을 초래합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지속가능성']}


In [47]:
print(type(response))

<class 'dict'>
